# ADIC Protocol Interactive Exploration
This notebook provides an interactive environment for exploring the ADIC consensus protocol

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from adic_simulator import AdicSimulator, AdicParams, QpDigits
from performance_analysis import PerformanceAnalyzer
import ipywidgets as widgets
from IPython.display import display, clear_output
%matplotlib inline

## 1. Basic Simulation

In [ ]:
# Create simulator with default parameters
params = AdicParams()
sim = AdicSimulator(params)

# Run simulation
sim.simulate(steps=50, verbose=True)

# Display metrics
metrics = sim.analyze_metrics()
pd.DataFrame([metrics]).T

## 2. Visualize DAG Structure

In [ ]:
# Visualize the DAG
sim.visualize_dag(max_nodes=30)

## 3. Interactive Parameter Exploration

In [ ]:
# Interactive widgets for parameter tuning
def run_interactive_simulation(k, q, d, rho_0, rho_1, rho_2, steps):
    clear_output(wait=True)
    
    # Create parameters
    params = AdicParams(
        k=k,
        q=q,
        d=d,
        rho=[rho_0, rho_1, rho_2]
    )
    
    # Run simulation
    sim = AdicSimulator(params)
    sim.simulate(steps=steps, verbose=False)
    
    # Display results
    metrics = sim.analyze_metrics()
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Metrics bar chart
    ax1 = axes[0]
    keys = ['tips', 'finalized', 'total_messages']
    values = [metrics[k] for k in keys]
    ax1.bar(keys, values, color=['red', 'green', 'blue'])
    ax1.set_title('Message Statistics')
    ax1.set_ylabel('Count')
    
    # Finalization rate over time
    ax2 = axes[1]
    ax2.pie([metrics['finalized'], metrics['total_messages'] - metrics['finalized']], 
            labels=['Finalized', 'Pending'],
            colors=['green', 'orange'],
            autopct='%1.1f%%')
    ax2.set_title('Finalization Status')
    
    # Depth distribution
    ax3 = axes[2]
    depths = [m.depth for m in sim.messages.values()]
    ax3.hist(depths, bins=20, color='skyblue', edgecolor='black')
    ax3.set_title('Depth Distribution')
    ax3.set_xlabel('Depth')
    ax3.set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed metrics
    print(f"\nDetailed Metrics:")
    print(f"  Finalization Rate: {metrics['finalization_rate']:.2%}")
    print(f"  Average Depth: {metrics['avg_depth']:.1f}")
    print(f"  Max Depth: {metrics['max_depth']}")
    print(f"  DAG Diameter: {metrics['dag_diameter']}")
    print(f"  Connected Components: {metrics['connected_components']}")

# Create interactive widgets
interact = widgets.interactive(
    run_interactive_simulation,
    k=widgets.IntSlider(min=5, max=50, step=5, value=20, description='k-core:'),
    q=widgets.IntSlider(min=1, max=10, step=1, value=3, description='q (diversity):'),
    d=widgets.IntSlider(min=1, max=10, step=1, value=3, description='d (degree):'),
    rho_0=widgets.IntSlider(min=1, max=5, step=1, value=2, description='ρ[0]:'),
    rho_1=widgets.IntSlider(min=1, max=5, step=1, value=2, description='ρ[1]:'),
    rho_2=widgets.IntSlider(min=1, max=5, step=1, value=1, description='ρ[2]:'),
    steps=widgets.IntSlider(min=10, max=200, step=10, value=50, description='Steps:')
)

display(interact)

## 4. P-adic Mathematics Visualization

In [ ]:
# Visualize p-adic balls
from adic_simulator import vp_diff, proximity_score

p = 3
values = range(0, 27)  # 0 to 3^3-1

# Create p-adic representations
qp_values = [QpDigits(v, p, precision=5) for v in values]

# Compute distance matrix
n = len(qp_values)
distance_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        distance_matrix[i, j] = p ** (-vp_diff(qp_values[i], qp_values[j]))

# Visualize distance matrix
plt.figure(figsize=(10, 8))
plt.imshow(distance_matrix, cmap='viridis_r', interpolation='nearest')
plt.colorbar(label='P-adic Distance')
plt.title(f'P-adic Distance Matrix (p={p})')
plt.xlabel('Value')
plt.ylabel('Value')

# Add grid for balls
for radius in [3, 9, 27]:
    plt.axhline(radius-0.5, color='red', linestyle='--', alpha=0.5)
    plt.axvline(radius-0.5, color='red', linestyle='--', alpha=0.5)

plt.show()

# Show ball structure
print("Ball Structure (radius=1):")
balls_r1 = {}
for v, qp in zip(values, qp_values):
    ball = qp.ball_id(1)
    if ball not in balls_r1:
        balls_r1[ball] = []
    balls_r1[ball].append(v)

for ball, members in sorted(balls_r1.items()):
    print(f"  Ball {ball}: {members}")

## 5. Performance Analysis

In [ ]:
# Run performance analysis
analyzer = PerformanceAnalyzer()

# Test different k values
k_values = [5, 10, 15, 20, 25, 30]
results = []

for k in k_values:
    params = AdicParams(k=k)
    perf = analyzer.benchmark_throughput(params, duration=5.0)
    results.append({
        'k': k,
        'throughput': perf['throughput'],
        'acceptance_rate': perf['acceptance_rate'],
        'finalized': perf['finalized']
    })

df = pd.DataFrame(results)

# Plot results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(df['k'], df['throughput'], 'b-o')
axes[0].set_xlabel('k-core parameter')
axes[0].set_ylabel('Throughput (msg/s)')
axes[0].set_title('Throughput vs k')
axes[0].grid(True)

axes[1].plot(df['k'], df['acceptance_rate'], 'g-s')
axes[1].set_xlabel('k-core parameter')
axes[1].set_ylabel('Acceptance Rate')
axes[1].set_title('Acceptance Rate vs k')
axes[1].grid(True)

axes[2].plot(df['k'], df['finalized'], 'r-^')
axes[2].set_xlabel('k-core parameter')
axes[2].set_ylabel('Finalized Messages')
axes[2].set_title('Finalization vs k')
axes[2].grid(True)

plt.tight_layout()
plt.show()

print("\nPerformance Summary:")
print(df.to_string())

## 6. Attack Simulation

In [ ]:
# Simulate network with adversarial nodes
class AdversarialSimulator(AdicSimulator):
    def __init__(self, params, adversarial_fraction=0.2):
        super().__init__(params)
        self.adversarial_fraction = adversarial_fraction
        self.adversarial_nodes = set()
    
    def simulate_adversarial_step(self):
        """Simulate step with potential adversarial behavior"""
        is_adversarial = np.random.random() < self.adversarial_fraction
        
        if is_adversarial:
            # Adversarial behavior: try to create conflicting messages
            author = f"adversary_{np.random.randint(1, 5)}"
            self.adversarial_nodes.add(author)
            
            # Try to select parents that violate constraints
            if self.tips:
                # Select random parents (potentially violating diversity)
                parents = list(np.random.choice(list(self.tips), 
                                              size=min(2, len(self.tips)), 
                                              replace=False))
            else:
                parents = []
        else:
            # Normal behavior
            return self.simulate_step()
        
        # Create adversarial message
        features = [QpDigits(np.random.randint(0, 100), self.params.p) 
                   for _ in range(len(self.params.rho))]
        
        msg_id = self.generate_message_id()
        message = Message(
            id=msg_id,
            parents=parents,
            features=features,
            timestamp=time.time(),
            author=author,
            content=f"Adversarial message from {author}",
            reputation=0.1  # Low reputation
        )
        
        # Check if it passes admissibility (it shouldn't)
        admissible, reason = self.check_admissibility(message)
        if not admissible:
            return None  # Rejected as expected
        else:
            # If it passes, add it
            self.add_message(message)
            return message

# Run adversarial simulation
from adic_simulator import Message
import time

adv_sim = AdversarialSimulator(AdicParams(), adversarial_fraction=0.3)
adv_sim.create_genesis()

honest_accepted = 0
adversarial_accepted = 0
total_rejected = 0

for _ in range(100):
    if np.random.random() < 0.3:
        msg = adv_sim.simulate_adversarial_step()
        if msg:
            adversarial_accepted += 1
        else:
            total_rejected += 1
    else:
        msg = adv_sim.simulate_step()
        if msg:
            honest_accepted += 1
        else:
            total_rejected += 1

print("Adversarial Simulation Results:")
print(f"  Honest messages accepted: {honest_accepted}")
print(f"  Adversarial messages accepted: {adversarial_accepted}")
print(f"  Total messages rejected: {total_rejected}")
print(f"  Security rate: {honest_accepted / (honest_accepted + adversarial_accepted):.2%}")
print(f"  Adversarial nodes identified: {len(adv_sim.adversarial_nodes)}")